In [1]:
import numpy as np, pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import normalize

from lightgbm import LGBMClassifier

import datetime

In [2]:
!ls data/

test.tsv  train.tsv


In [3]:
df = pd.read_csv('data/train.tsv', sep='\t')

In [4]:
target = df['1.53']
X = df[[c for c in df.columns if c != '1.53']]

# Select important features
M = pd.DataFrame(mutual_info_classif(X, target), index=X.columns)
picked = M.sort_values(by=0, ascending=False)[M[0]>0.000446].index
X_mutal = X[picked].copy()

/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [5]:
X_t = df[[c for c in df.columns if (float(c) < 0.8 and c != '1.53')]].copy()

In [6]:
X_norm = normalize(X)

In [7]:
def testing(X, target):
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25, random_state=42)
    clf = LGBMClassifier()
    print(pd.DataFrame(cross_validate(clf, X, target, 
                                      scoring=['neg_log_loss', 'roc_auc', 'f1', 'accuracy'], n_jobs=-1, cv=6)).mean())
    clf.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='binary_error',
        early_stopping_rounds=5)
    return clf

In [8]:
testing(X_mutal, target)
testing(X_t, target)

/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_neg_log_loss'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages

fit_time              7.748930
score_time            0.200202
test_neg_log_loss    -0.710070
train_neg_log_loss   -0.531822
test_roc_auc          0.493857
train_roc_auc         0.965696
test_f1               0.483322
train_f1              0.899241
test_accuracy         0.497651
train_accuracy        0.900330
dtype: float64
[1]	valid_0's binary_error: 0.4912
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_error: 0.488
[3]	valid_0's binary_error: 0.4904
[4]	valid_0's binary_error: 0.4976
[5]	valid_0's binary_error: 0.4856
[6]	valid_0's binary_error: 0.4964
[7]	valid_0's binary_error: 0.4808
[8]	valid_0's binary_error: 0.484
[9]	valid_0's binary_error: 0.4864
[10]	valid_0's binary_error: 0.4848
[11]	valid_0's binary_error: 0.4896
[12]	valid_0's binary_error: 0.4804
[13]	valid_0's binary_error: 0.4844
[14]	valid_0's binary_error: 0.49
[15]	valid_0's binary_error: 0.486
[16]	valid_0's binary_error: 0.4912
[17]	valid_0's binary_error: 0.49
Early stopping, be

/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_neg_log_loss'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [9]:
clf = testing(X_norm, target)

fit_time              7.810256
score_time            0.092356
test_neg_log_loss    -0.000673
train_neg_log_loss   -0.000041
test_roc_auc          1.000000
train_roc_auc         1.000000
test_f1               0.999899
train_f1              1.000000
test_accuracy         0.999900
train_accuracy        1.000000
dtype: float64
[1]	valid_0's binary_error: 0.022
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_error: 0.0004
[3]	valid_0's binary_error: 0.0004
[4]	valid_0's binary_error: 0.0004
[5]	valid_0's binary_error: 0.0004
[6]	valid_0's binary_error: 0.0004
[7]	valid_0's binary_error: 0.0004
Early stopping, best iteration is:
[2]	valid_0's binary_error: 0.0004


/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_neg_log_loss'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages

In [10]:
test = pd.read_csv('data/test.tsv', sep='\t')
test = normalize(test)

In [11]:
pd.DataFrame(clf.predict(test)).to_csv('result.tsv', sep='\t', index = False)

In [12]:
X_norm_l1 = normalize(X, norm='l1')

In [13]:
clf = testing(X_norm, target)

fit_time              8.664374
score_time            0.054641
test_neg_log_loss    -0.000673
train_neg_log_loss   -0.000041
test_roc_auc          1.000000
train_roc_auc         1.000000
test_f1               0.999899
train_f1              1.000000
test_accuracy         0.999900
train_accuracy        1.000000
dtype: float64
[1]	valid_0's binary_error: 0.022
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_error: 0.0004
[3]	valid_0's binary_error: 0.0004
[4]	valid_0's binary_error: 0.0004
[5]	valid_0's binary_error: 0.0004
[6]	valid_0's binary_error: 0.0004
[7]	valid_0's binary_error: 0.0004
Early stopping, best iteration is:
[2]	valid_0's binary_error: 0.0004


/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_neg_log_loss'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/denis/anaconda3/lib/python3.6/site-packages